$$
\newcommand{\L}{\mathcal{L}}
\renewcommand{\R}{\mathbb{R}}
\newcommand{\var}{\text{Var}}
\newcommand{\E}{\mathbb{E}}
$$

# `Материалы кафедры ММП факультета ВМК МГУ. Введение в глубокое обучение.`

# `Лекция 02. Регуляризация в DL`

### `Материалы составил Алексеев Илья (@voorhs)`

#### `Москва, Весенний семестр 2025`

## `Затухание градиентов`

На прошлом занятии мы вывели формулу для градиентов весов линейного (полносвязного) слоя:
$$
z_i=W_ia_{i-1}\Rightarrow
\begin{cases}
\nabla_{W_i}\L=(\nabla_{z_i}\L)a_{i-1}^T,\\
\nabla_{a_{i-1}}\L=W_i^T(\nabla_{z_i}\L).
\end{cases}
$$
А также градиенты функции активации в общем виде:
$$
a_i=f(z_i)\Rightarrow\nabla_{z_i}\L=(\nabla_{a_i}\L) {\partial a_i\over\partial z_i}.
$$
Нетрудно вывести формулу градиентов весов на $i$-ом слое ($i=\overline{1,L}$):
$$
\begin{align}
\nabla_{W_i}\L&=(\nabla_{z_i}\L)a_{i-1}^T\\
&=\left((\nabla_{a_i}\L) {\partial a_i\over\partial z_i}\right)a_{i-1}^T\\
&=\left((W_{i+1}^T(\nabla_{z_{i+1}}\L)) {\partial a_i\over\partial z_i}\right)a_{i-1}^T\\
&=\left(\left(W_{i+1}^T\left((\nabla_{a_{i+1}}\L){\partial a_{i+1}\over\partial z_{i+1}}\right)\right) {\partial a_i\over\partial z_i}\right)a_{i-1}^T\\
&=\ldots\\
&=\left(\prod_{j=i+1}^LW_{j}^T\right)(\nabla_{a_L}\L)\left(\prod_{j=i+1}^L{\partial a_{j}\over\partial z_j}\right)a_{i-1}^T.
\end{align}
$$
Эта формула является очень ценной для понимания того, в чем различие обучения первых слоев нейросети ($i=1,2,3,\ldots$) и последних ($i=L,L-1,L-2,\ldots$). Давайте проанализируем норму полученного градиента:
$$
\|\nabla_{W_i}\L\|\leqslant \left(\prod_{j={i+1}}^L\|W_j\|\right)\left(\prod_{j={i+1}}^L\left\|{\partial a_j\over\partial z_j}\right\|\right)\|a_{i-1}\|
$$
Рассмотрим случай, когда в качестве функции $a_i=f(z_i)$ выбрана сигмоида. Известно следующее:
$$
\begin{cases}
{\partial a_j\over\partial z_j}=a_i(1-a_i)\\
0<a_i<1
\end{cases}\Rightarrow0<\left\|{\partial a_j\over\partial z_j}\right\|<1\Rightarrow\prod_{j={i+1}}^L\left\|{\partial a_j\over\partial z_j}\right\|\xrightarrow{L\to\infty}0.
$$
То есть градиенты весов первых слоев будут значительно меньше градиентов последних слоев. Это означает две вещи:

- выход нейросети практически не зависит от преобразований, совершенных на первых слоях, т.е. “информация” не доходит до конца;
- во время оптимизации веса первых слоев будут меняться намного медленнее, чем веса последних слоев.

Данное явление называют затуханием градиентов.

## `Функции активации`

Одним из эффективных способов борьбы с затуханием градиентов является правильный выбор функции активации. Рассмотрим набор популярных вариантов с точки зрения их сопротивления затуханию градиентов и не только.

| Функция                                                      | Затухание | Тождество | Гладкость | Сложность |
| ------------------------------------------------------------ | --------- | --------- | --------- | --------- |
| $\sigma(z)=\dfrac{1}{1+e^{-z}}$                              | FAIL      | FAIL      | OK        | FAIL      |
| $\tanh(z)=2\sigma(2z)-1$                                     | FAIL      | OK        | OK        | FAIL      |
| $\text{ReLU}(z)=\max\{0,z\}$                                 | OK        | FAIL      | OK/FAIL   | OK        |
| $\text{LeakyReLU}(z)=\begin{cases}z, & z\geqslant0,\\\alpha z,& \text{otherwise},\end{cases}$ | OK        | OK        | OK/FAIL   | OK        |
| $\text{ELU}(z)=\begin{cases}z, & z\geqslant0,\\e^z-1,& \text{otherwise}.\end{cases}$ | OK        | OK        | OK        | FAIL      |
| $\text{GELU}(z)=z\cdot\Phi(z)$                               | ОК        | ОК        | ОК        | FAIL      |
| $\text{Swish}(z)=z\cdot\sigma(z)$                            | ОК        | ОК        | ОК        | FAIL      |
| $\text{Mish}(z)=z\cdot\tanh(\text{softplus}(z))$             | ОК        | ОК        | ОК        | FAIL      |
| $\text{ReLU6}(z)=\min(\max(0,z),6)$                          | ОК        | FAIL      | ОК/FAIL   | OK        |

### Тождественное преобразование

Нейросеть с сигмоидной функцией активации нельзя научить тождественному преобразованию, т.к. вход сигмоиды — это $\R$, а выход — это отрезок $[0,1]$. Поддерживать тождественное преобразование — это разумное требование к нейросети, т.к. иногда лучшее, что может сделать слой — это просто не испортить имеющиеся признаки.

### Вычислительная сложность

Сложные функции по типу синусов, экспонент и логарифмов на компьютере считаются с помощью рядов Тейлора. При большом числе слоев, число операций, требуемых для вычисления функций активации может сильно возрасти. Поэтому например выбор $\text{ReLU}$ вместо $\sigma$ может заметно ускорить проход по нейросети.

### Гладкость

Функцию называют гладкой, если её производная является непрерывной функцией. Для таких функций методы оптимизации сходятся на порядок быстрее, поэтому на первый взгляд кажется, что $\text{ReLU},\text{LeakyReLU}$ следует сразу вычеркнуть из списка. Но поскольку свойство гладкости нарушается лишь в одной точке, на практике методы гладкой оптимизации работают и для них. Существует даже термин: дифференцируемые в смысле нейросетей функции.

## `Нормализация`

Посмотрим на проблему затухающих градиентов с другой стороны. Обратим внимание на то, что если вход сигмоиды большой по модулю (очень отрицательный или очень положительный), то выход соответствует тем участкам графика, где функция очень близка к константной. Именно в этих областях числовой прямой производная близка к нулю. Поэтому для сигмоидной функции очень желательно, чтобы входы были “около нуля”.

![](pics/sigmoid.svg)

На самом деле, это желательно для любой функции активации и для любой нейросети. Например, если входы ReLU всегда будут отрицательными, то “сигнал” полностью оборвется. Если же они все будут положительными, то пропадает нелинейность, которую должна вносить функция активации.

Привести входы слоя к промежутку “около нуля” позволяет нормализация. Вспомним, как работала нормализация признаков в классическом машинном обучении. Пусть имеется батч данных, для которых посчитаны входные активации $a\in\R^{B\times d}$ какого-то промежуточного полносвязного слоя. Мы хотим нормализовать эти активации перед подачей в этот полносвязный слой.

1. Считаем выборочные среднее и дисперсию для каждого признака по батчу:

$$
\overline\mu_{j}={1\over B}\sum_{i=1}^Ba_{ij},\quad\overline\sigma_j^2={1\over B}\sum_{i=1}^B(a_{ij}-\overline\mu_j)^2,\quad j=\overline{1,d}.
$$

2. Нормализуем данные:

$$
\tilde a_{ij}={a_{ij}-\overline\mu_j\over\overline\sigma_j+\varepsilon},\quad i=\overline{1,B},\ j=\overline{1,d},\ \varepsilon>0.
$$

Константа $\varepsilon$ используется на всякий случай во избегание деления на ноль.

Описанная конструкция применяется в классическом машинном обучении, но не пригодна для глубокого обучения по следующей причине. Размер батча $B$ как правило не так велик, чтобы получить точные оценки $\overline\mu,\overline\sigma$. Это приводит к тому, что от батча к батчу они будут сильно меняться и слой не обучится должным образом, т.к. в данных теряется много информации.

Решить проблему “скачущих” статистик можно довольно простым способом — использовать скользящее среднее. Алгоритм нормализации будет следующий. Для $j=\overline{1,d}$:

0. В начале обучения заводим $\hat\mu_j=0$, $\hat\sigma_j=1$
1. На пришедшем батче данных считаем $\overline\mu_j$, $\overline\sigma_j$
2. Обновляем статистики (как правило $\alpha=0.9$):

$$
\begin{align}
\hat\mu_j&:=\alpha\hat\mu+(1-\alpha)\hat\mu_j\\
\hat\sigma_j^2&:=\alpha\hat\sigma_j^2+(1-\alpha)\hat\sigma_j^2,
\end{align}
$$

3. Нормализуем данные.

Описанный алгоритм уже хорош! Но стоит помнить, что даже со сглаживанием размер батча все равно должен быть достаточно большим.

Данный алгоритм не дотягивает до того, что обычно называют Batch Normalization. Не хватает добавить один шаг в конце:
$$
\widehat a_{ij}=\gamma_j\tilde a_{ij} + \delta_j,
$$
где $\gamma_j,\delta_j\in\R$ — обучаемые параметры. Данный шаг мотивирован двумя соображениями:

1. Обучаемое преобразование может внести корректировку в “искажение”, которое внесено нормализацией.
2. Существуют такие $\gamma,\delta$, что $\widehat a_{ij}=a_{ij}$, т.е. возможно выучить тождественное преобразование.

Батч нормализация является эффективным способом борьбы с затуханием градиентов в результате насыщения функций активации.

Стоит упомянуть, что помимо batch norm применяется layer norm. Разница состоит в том, что среднее и дисперсия считаются не вдоль размерности $B$, а вдоль размерности $d$. Такой метод применяется по ряду причин. Например, если вычислительные ресурсы не позволяют обрабатывать достаточно большой батч, чтобы посчитанные статистики были достаточно точными.

## `Инициализация весов`

Вернемся к выражению для нормы градиентов весов:
$$
\|\nabla_{W_i}\L\|\leqslant \underbrace{\left(\prod_{j={i+1}}^L\|W_j\|\right)}_{\text{init.}}\underbrace{\left(\prod_{j={i+1}}^L\left\|{\partial a_j\over\partial z_j}\right\|\right)}_\text{act.fun., norm.}\underbrace{\|a_{i-1}\|}_\text{norm.}
$$
Проанализируем методы, которые мы успели ввести выше.

- Выбор функции активации влияет на второй множитель справа. 
- Нормализация направлена на второй и третий множитель справа.

Как повлиять на первый множитель? Если нормы матриц будут слишком маленькими, то градиент затухнет, а если большими, — то взорвется. Чтобы побороть это, необходимо выбрать правильную инициализацию весов.

### `Инициализация Xavier Glorot`

Пусть $z=Wx$, где $W\in\R^{n_\text{in}\times n_\text{out}}$. Знаем, что $\L'_x=W^T\L'_z$. Допустим, на вход подается нормированная выборка:
$$
x_i\stackrel{iid}\sim N(0,\var(x))
$$
Найдем самый разумный способ инициализировать веса $w_{ij}$. Будем искать среди нормальных распределений с неизвестным параметром $\var(w)$:
$$
w_{ij}\stackrel{iid}\sim N(0,\var(w)).
$$
При проходе вперед мы хотели бы сохранить нормированность выборки. Заметим, что при проходе вперед

- сохраняется центрированность выборки:

$$
\E z_i=\E\bigg(\sum_j w_{ij} x_j\bigg)=\sum_j(\E w_{ij})(\E x_j)=\{\E w_{ij}=\E x_j=0\}=0.
$$

- изменяется дисперсия выборки:

$$
\var(z)=\var\bigg(\sum_j w_{ij}x_j\bigg)=\sum_j\var(w_{ij})\var(x_j)=n_\text{in}\var(w)\var(x).
$$

При проходе назад мы бы хотели сохранить нормированность градиентов, т.к. это сохранит нормированность выборки. Заметим, что при проходе назад

- сохраняется центрированность градиентов:

$$
\E\left({\L'_{x_i}}\right)=\E\bigg(\sum_jw_{ji}\L'_{z_j}\bigg)=\sum_j(\E w_{ji})\E \bigg(\L'_{z_j}\bigg)=\{\E w_{ij}=0\}=0
$$

- изменяется дисперсия:

$$
\var\left(\L'_{x_i}\right)=n_\text{out}\var(w)\var\left(L'_{z_j}\right).
$$

Значит, центрированность всегда сохраняется, осталось потребовать, чтобы не менялась дисперсия. Т.е.
$$
\var(z)=\var(x),\quad \var\left(\L'_x\right)=\var\left(\L'_z\right).
$$
Получили систему относительно $\var(w)$.
$$
\begin{cases}
n_\text{in}\var(w)=1, \\
n_\text{out}\var(w)=1.
\end{cases}
$$
Эта система несовместна, но в качестве компромисса можем взять
$$
\var(w)={2\over n_\text{in}+n_\text{out}}.
$$
Это первый вариант инициализации. Второй состоит в том, что использовать не нормальное, а равномерно распределение, которое ищется из тех же соображений:
$$
w_{ij}\stackrel{iid}\sim \text{Unif}\left[-{1\over\sqrt n},{1\over\sqrt n}\right],\quad n={n_\text{in}+n_\text{out}\over6}.
$$

### `Инициализация для tanh`

Рассмотрим функцию активации удовлетворяющую следующим условиям:
1. Нечётность, то есть $g(x) = -g(-x)$
2. $g^{\prime}(0) = 1$

Так, этим условиям удовлетворяет Tanh.

Наша цель избежать затухания градиентов в процессе обучения за счёт правильной начальной инициализации.

Рассмотрим модель нейросети — MLP с функцией активации $g$. Положим $a^{L}, b^{L} \in \mathbb{R}^{n^{L} \times 1}$ — активации и вектор сдвига слоя $L$, $W^{L} \in \mathbb{R}^{n^{L}\times n^{L-1}}$ -- веса слоя $L$, $x \in \mathbb{R}^{n^{0}}$ — вектор входных данных.
$$
a^{0} = x
$$
$$
z^{L} = W^{L}a^{L-1} + b^{L}
$$
$$
a^{L} = g(z^{L})
$$

Сделаем следующие предположения:
1. Все вектора сдвига равны нулю: $b^{L} = 0 \; \forall L$
2. В начале обучения модель находится в *линейном режиме*, то есть $z^{L}_{j} \approx 0 \Rightarrow f(z^{L}) \approx z^{L}$
3. $W^{L}_{jk}, a^{L}_{j}, z^{L}_{j}, x_{j}, \delta^{L}_{j} = \frac{\partial\mathfrak{L}}{\partial z^{L}_{j}}$ — распределены независимо, при этом распределение каждой группы переменных для одного слоя зависит только от номера этого слоя и одинаково для всех переменных.
4. $\mathbb{E}x = \mathbb{E}x_{j} = 0, \mathbb{D}x = \mathbb{D}x_{j} = 1$

В этих предположениях рассмотрим прямой и обратный проход по сети. Рассмотрим, как меняется распределения активаций от слоя к слою.

1. Прямой проход

На прямом проходе мы хотим добиться того, чтобы матожидание и дисперсия активаций оставались неизменными от слоя к слою. Тем самым мы гарантируем что активации не будут "взрываться" или "испаряться" к последним слоям сети.

Очевидно, что из второго предположения следует, что матожидания всех активаций равно 0. Рассмотрим дисперсию активаций:
$$
\mathbb{D}z^{L} = \mathbb{D}z^{L}_{j} = \mathbb{D} \sum\limits_{i=1}^{n^{L-1}} W^{L}_{jk}a^{L-1}_{k} = \sum\limits_{i=1}^{n^{L-1}} \mathbb{D}W^{L}_{jk}\mathbb{D}a^{L-1}_{k} = n^{L-1}\mathbb{D}W^{L}\mathbb{D}a^{L-1}
$$
Учитывая второе условие:
$$
\mathbb{D}z^{L} = n^{L-1}\mathbb{D}W^{L}\mathbb{D}a^{L-1} = n^{L-1}\mathbb{D}W^{L}\mathbb{D}z^{L-1} = n^{L-1}\mathbb{D}W^{L}n^{L-2}\mathbb{D}W^{L-1}\mathbb{D}z^{L-2} = ... = \mathbb{D}x\prod\limits_{l=0}^{L-1}n^{l}\mathbb{D}W^{l+1}
$$

$$
\mathbb{D}z^{L} = \mathbb{D}z^{L-1} \iff n^{L - 1}\mathbb{D}W^{L} = 1 \iff \mathbb{D}W^{L} = \frac{1}{n^{L-1}} \;\;\;\; (6)
$$

2. Обратный проход

На прямом проходе мы хотим добиться того, чтобы матожидание и дисперсия градиентов оставались неизменными от слоя к слою. Тем самым мы гарантируем что градиенты не будут "взрываться" или "испаряться" к первым слоям сети.

$$
\delta^{L}_{j} = \sum\limits_{k=1}^{n^{L+1}}\delta^{L+1}_{k}\frac{\partial z^{L+1}_{k}}{\partial z^{L}_{j}} = \sum\limits_{k=1}^{n^{L+1}}\delta^{L+1}_{k}\frac{\partial }{\partial z^{L}_{j}}\sum\limits_{i=1}^{n^{L}}W^{L+1}_{ki}g(z^{L_{i}}) = \sum\limits_{k=1}^{n^{L+1}}\delta^{L+1}_{k}W^{L+1}_{kj}\frac{\partial g(z^{L_{j}})}{\partial z^{L}_{j}} = \sum\limits_{k=1}^{n^{L+1}}\delta^{L+1}_{k}W^{L+1}_{kj}g^{\prime}(z^{L_{j}})
$$

Учитывая, что $g^{\prime}(x) \approx 1$

$$
\mathbb{E}\delta^{L} = \mathbb{E}\delta^{L}_{j} = n^{L+1}\mathbb{E}W^{L+1}\mathbb{E}\delta^{L+1} = 0 \Leftarrow \mathbb{E}W^{L} = 0
$$

$$
\mathbb{D}\delta^{L} = \mathbb{D}\delta^{L}_{j} = n^{L+1}\mathbb{D}W^{L+1}\mathbb{D}\delta^{L+1} = n^{L+1}\mathbb{D}W^{L+1}n^{L+2}\mathbb{D}W^{L+2}\mathbb{D}\delta^{L+2} = ... = \mathbb{D}\delta^{d}\sum\limits_{m=L+1}^{d-1}n^{m}\mathbb{D}W^{m}
$$


$$
\mathbb{D}\delta^{L} = \mathbb{D}\delta^{L-1} \iff n^{L}\mathbb{D}W^{L} = 1 \iff \mathbb{D}W^{L} = \frac{1}{n^{L}} \;\;\;\; (7)
$$

Заметим, $(6), (7)$ могут выполняться одновременно тогда и только тогда, когда $n^{L} = n^{L+1} \; \forall L$, то есть когда ширина MLP остаётся постоянной. В ином случае, предлагается брать среднее гармоническое полученных оценок:
$$
\begin{cases} 
    \mathbb{E}W^{L} = 0\\
    \mathbb{D}W^{L} = \frac{2}{n^{L-1} + n^{L}} = \frac{2}{n^{\text{in}} + n^{\text{out}}}
\end{cases} \;\;\;\;\; (8)
$$

В зависимости от того, что важнее — сохранение дисперсии активаций или сохранение дисперсий градиентов можно использовать или (6), или (7), но обычно, дисперсия градиентов важнее.

Существует множество распределений, для которых $(8)$ выполняется. Стандартным вариантом является использование равномерного распределения на подходящем интервале:
$$
W^{L}_{ij} \sim \mathcal{U}[-\sqrt{\frac{6}{n^{\text{in}} + n^{\text{out}}}}, \sqrt{\frac{6}{n^{\text{in}} + n^{\text{out}}}}]
$$

Аналогичные результаты могут быть получены для других функций активации — ReLU, Sigmoid. 

### `Ортогональная инициализация`

Предыдущий метод инициализации основан на идее сохранения дисперсии и среднего. Следующий вывод

Инициализируем веса $W\in\R^{n\times p}$ так, чтобы столбцы $W$ были ортогональны, т.е. $W^TW=I$. Тогда
$$
\begin{align}
& \|z\|_2=\|Wx\|_2=\|x\|_2,\\
& \|\nabla_x\L\|_2=\|W^T\nabla_z\L\|_2=
\|\nabla_z\L\|_2.
\end{align}
$$
Т.е. нормы сохраняются. 

Как сохранить ортогональность матрицы $W$ во время оптимизации? При оптимизации градиентными методами, матрица $W$ выйдет из множества ортогональных при первом же шаге. Чтобы предотвратить это, используют прием под названием репараметризация.

Справедливо следующее свойство. Для случайной $V\in\R^{d\times d}$, матрица $W=\exp(V-V^T)$ является ортогональной, где $\text{exp}$ — матричная экспонента.

## `Skip-connection / Residual Connection / DenseNet`

Еще одним способом регуляризации нейросетей является “прокидывание” связей, или использование остаточных связей. Идея простая: давайте переиспользовать признаки с предыдущих слоев.

Переиспользовать признаки можно разными способами:

1. Поэлементно прибавлять предыдущие активации к текущим.
2. Конкатенировать предыдущие активации к текущим.

Такое прокидывание связей помогает решить несколько проблем:

- затухание градиентов, поскольку через прокинутую связь градиент “протекает” без изменений, т.к. это тождественное преобразование
- потеря признаков, потому что каждый линейный слой получает совершенно новые признаки; это небережливо по отношению к признакам с глубоких слоев, которые могли оказаться полезными
- устранение симметрии полносвязных слоев; подробнее об этой гипотезе смотрите здесь: https://habr.com/ru/articles/688350/
- сглаживание ландшафта функции потерь; подбобнее смотрите здесь: https://arxiv.org/abs/1712.09913

![](pics/skip-connection.png)

## `Drop Out`

Идея: на этапе обучения в каждом слое будем “отключать” случайно выбранные нейроны, т.е.
$$
\begin{align}
& z=Wx+b \\
& a:=g(z) \\
& y_i\sim\text{Bern}(1-p) \\
& a^\text{almost DO} := a\odot y
\end{align}
$$
Но в таком случае $\E a^\text{DO}=(1-p)\E a$. Поэтому dropout-значение нужно поделить на $1-p$:
$$
\begin{align}
& z=Wx+b \\
& a:=g(z) \\
& y_i\sim\text{Bern}(1-p) \\
& a^\text{DO} := a\odot y/(1-p)
\end{align}
$$
Авторы метода утверждают, что такая процедура исключает эффект коадаптации: когда верхние слои просто копируют предыдущие и ничему не учатся. “Выключение” случайных входных нейронов приводит к тому, что утилизируются все нейроны какие только есть, т.е. через них текут градиенты.

## `Стохастическая оптимизация`

При обучении нейросетей используются градиентные методы, причем стохастические. Оказывается, что это объясняется не только оптимизацией вычислительных ресурсов (избегание вычисления точного градиента по всей выборке), но и эффектом регуляризации, который привносит “шумный” градиент.

Рассмотрим на примере оптимизации MSE в задаче регрессии по одному признаку методом SGD с размером батча равным одному объекту. MSE выглядит так:
$$
\L={1\over \ell}\sum_{i=1}^\ell(wx_i-y)^2,
$$
где $y_i\in\R$ — истинная метка, $w\in\R$ — обучаемый вес, $x_i\in\R$ — единственный признак. Каждое слагаемое является параболой относительно $x_i$. Мы можем представить процесс оптимизации методом SGD следующим образом: на каждом шаге случайно выбирается одна парабола, делается шаг $\eta\cdot x_i(wx_i-y)$, который направлен в сторону вершины этой параболы (т.к. это оптимум для нее). Если мы верим, что задача регрессии имеет красивое решение, то вершины всех парабол находятся примерно около друг друга и есть какое-то “среднее” оптимальное место. Когда значение $w$ далеко от этого места, SGD безошибочно идет в его сторону. Но когда SGD приближается к оптимуму, есть вероятность, что случайно выбранная парабола будет в направлении, противоположном оптимуму. Поэтому SGD никогда не сойдется к оптимуму, вместо этого он будет оссциллировать в регионе неопределенности.

Это свойство SGD является недостатком с точки зрения методов оптимизации. Но мы решаем не просто задачу оптимизации, а задачу машинного обучения, в которой как правило запрещено идеально подгоняться под обучающую выборку. С этой точки зрения SGD вносит регуляризацию.

Это свойство особенно полезно в контексте глубокого обучения, потому что для многослойных сетей ландшафт функции потерь представляет собой минное поле с разбросанными минами — плохими локальными оптимумами. Стохастическая оптимизация позволяет обходить узкие локальные оптимумы, в то время как точный градиент просто ищет любой ближайший.
